In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pandas
import urllib
import re

from selenium import webdriver
from time import sleep
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

from selenium.common.exceptions import NoSuchElementException, WebDriverException

import csv
import time

from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# #A quick look at https://www.leafly.com/explore/page-61/sort-alpha shows 
# #that this is the last page of strains
# pages = 61 
# websites = []
# strain_names = []

# for i in range(pages+1):
#     page_url = "https://www.leafly.com/explore/page-"+str(i)+"/sort-alpha"
#     html = requests.get(page_url).text
#     soup = BeautifulSoup(html,'html.parser')
#     urls = soup.findAll("a",{"class":"ga_Explore_Strain_Tile"})
        
#     for url in urls:
#         strain_name = url.get('href')
#         strain_names.append(strain_name)
#         website = ("https://www.leafly.com" + strain_name)
#         websites.append(website)

In [ ]:
# #create url df
# url_df = pd.DataFrame(websites, columns = ['url'])

In [ ]:
# #create url csv
# url_df.to_csv('urls.csv')

In [ ]:
# #create strain_names csv
# strains_df = pd.DataFrame(strain_names, columns = ['strain_name'])

In [ ]:
# #create strain_names csv
# strains_df.to_csv('strains.csv')

In [ ]:
# #parse strain_name:
# parsed =[]
# for words in strains_df['strain_name']:
#     parse = re.findall(r"[\w']+", words)
#     parsed.append(parse)

In [ ]:
# #create name and strain df

# strain_type_df = pd.DataFrame(parsed, columns = ['strain', 'name1', 'name2',
#                                     'name3', 'name4', 'name5', 'name6'])

In [ ]:
# #drop last 5 columns - just fillers
# strain_type_df.drop(df.tail(5).index,inplace=True)

In [ ]:
# #merge name columns
# strain_type_df['name'] = strain_type_df[strain_type_df.columns[1:]].apply(
#     lambda x: ' '.join(x.dropna().astype(str)),
#     axis=1)

In [ ]:
# #drop unnecessary columns
# strain_type_df.drop(['name1', 'name2', 'name3', 'name4', 'name5', 'name6'], axis=1, inplace=True)

In [ ]:
# strain_type_df.to_csv('strain_type.csv')

In [ ]:
# #number of duplicates
# df['name'].duplicated().sum()

In [ ]:
# url_df['url'].duplicated().sum()

In [ ]:
# url_df.drop_duplicates(inplace=True)

In [ ]:
# url_df.reset_index()

In [ ]:
# len(url_df)

In [ ]:
# #breaking down scraping into smaller buckets

# urls_to_200 = urls_df['url'][:200]          #soups
# urls_to_700 = urls_df['url'][200:700]       #soups_to_700
# urls_to_1200 = urls_df['url'][700:1200]     #soups_to_1200
# urls_to_1700 = urls_df['url'][1200:1700]    #soups_to_1700
# urls_to_2200 = urls_df['url'][1700:2200]    #soups_to_2200
# urls_to_2700 = urls_df['url'][2200:2700]    #soups_to_2700
# urls_to_end = urls_df['url'][2700:]         #soups_to_end

# #scraping all at once
# urls_total = urls_df['url']                 #soups_total

In [ ]:
# #scrape individual strain pages

# soups_total = []

# for url in urls_total:
#     html = requests.get(url).text
#     soup = BeautifulSoup(html,'html.parser')
#     soups_total.append(soup)
#     time.sleep(1)

In [ ]:
# with open("soups_total.html", "w") as file:
#     file.write(str(soups_total))

### Get Features From Tags

In [ ]:
# #use this one

# #collect positive effects

# effect = []
# filename1 = "effect.csv"

# for i in tqdm(range(0, len(soups_total))): 
#     names = []
#     cont = soups_total[i].find("a",{"class":"active"})  
#     names.append(cont)
#     try:
#         for name in names:
#             strain_name = name.get('href') 
#             strain = strain_name.split('/')
#         div = soups_total[i].find('div', {'id': 'effects-tab-content'})
#         first_child = div.findChildren('div', {'class': "histogram-label"})
#         try:
#             if len(first_child) == 5:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text, 
#                     first_child[2].text, first_child[3].text, 
#                         first_child[4].text]
#                 par = {key:values}
            
#             elif len(first_child) == 4:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text, 
#                           first_child[2].text, first_child[3].text]
#                 par = {key:values}
            
#             elif len(first_child) == 3:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text, 
#                           first_child[2].text]
#                 par = {key:values}
            
#             elif len(first_child) == 2:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text]
#                 par = {key:values}
            
#             elif len(first_child) == 1:
#                 key = strain[2]
#                 values = [first_child[0].text]
#                 par = {key:values}
            
#             else:
#                 key = strain[2]
#                 values = None
#                 par = {key:values}       
#         except:
#             key = strain[2]
#             values = None
#             par = {key:values}
#     except:
#         key = strain[2]
#         values = None
#         par = {key:values}
    
        
#     effect.append(par)

# df_effect = pd.DataFrame(effect)
# df_effect.to_csv(filename1)

# #collect medical attributes

# med = []
# filename2 = "med.csv"

# for i in tqdm(range(0, len(soups_total))): 
#     names = []
#     cont = soups_total[i].find("a",{"class":"active"})  
#     names.append(cont)
#     try:
#         for name in names:
#             strain_name = name.get('href') 
#             strain = strain_name.split('/')
#         div = soups_total[i].find('div', {'id': 'medical-tab-content'})
#         first_child = div.findChildren('div', {'class': "histogram-label"})
#         try:
#             if len(first_child) == 5:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text, 
#                     first_child[2].text, first_child[3].text, 
#                         first_child[4].text]
#                 par = {key:values}
            
#             elif len(first_child) == 4:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text, 
#                           first_child[2].text, first_child[3].text]
#                 par = {key:values}
            
#             elif len(first_child) == 3:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text, 
#                           first_child[2].text]
#                 par = {key:values}
            
#             elif len(first_child) == 2:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text]
#                 par = {key:values}
            
#             elif len(first_child) == 1:
#                 key = strain[2]
#                 values = [first_child[0].text]
#                 par = {key:values}
            
#             else:
#                 key = strain[2]
#                 values = None
#                 par = {key:values}       
#         except:
#             key = strain[2]
#             values = None
#             par = {key:values}
#     except:
#         key = strain[2]
#         values = None
#         par = {key:values}
    
        
#     med.append(par)

# df_med = pd.DataFrame(med)
# df_med.to_csv(filename2)

# #collect negative attributes

# neg = []
# filename3 = "neg.csv"

# for i in tqdm(range(0, len(soups_total))): 
#     names = []
#     cont = soups_total[i].find("a",{"class":"active"})  
#     names.append(cont)
#     try:
#         for name in names:
#             strain_name = name.get('href') 
#             strain = strain_name.split('/')
#         div = soups_total[i].find('div', {'id': 'negatives-tab-content'})
#         first_child = div.findChildren('div', {'class': "histogram-label"})
#         try:
#             if len(first_child) == 5:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text, 
#                     first_child[2].text, first_child[3].text, 
#                         first_child[4].text]
#                 par = {key:values}
            
#             elif len(first_child) == 4:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text, 
#                           first_child[2].text, first_child[3].text]
#                 par = {key:values}
            
#             elif len(first_child) == 3:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text, 
#                           first_child[2].text]
#                 par = {key:values}
            
#             elif len(first_child) == 2:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text]
#                 par = {key:values}
            
#             elif len(first_child) == 1:
#                 key = strain[2]
#                 values = [first_child[0].text]
#                 par = {key:values}
            
#             else:
#                 key = strain[2]
#                 values = None
#                 par = {key:values}       
#         except:
#             key = strain[2]
#             values = None
#             par = {key:values}
#     except:
#         key = strain[2]
#         values = None
#         par = {key:values}
    
        
#     neg.append(par)

# df_neg = pd.DataFrame(neg)
# df_neg.to_csv(filename3)

# #collect flavors

# flavors = []
# filename4 = "flavors.csv"

# for i in tqdm(range(0, len(soups_total))): 
#     names = []
#     cont = soups_total[i].find("a",{"class":"active"})  
#     names.append(cont)
#     try:
#         for name in names:
#             strain_name = name.get('href')
#             strain = strain_name.split('/')
#         first_child = soups_total[i].findAll('div',attrs={"class" : "flavor-name"})
#         try:
#             if len(first_child) == 3:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text, 
#                         first_child[2].text]
#                 par = {key:values}
#             elif len(first_child) == 2:
#                 key = strain[2]
#                 values = [first_child[0].text, first_child[1].text]
#                 par = {key:values}
#             elif len(first_child) == 1:
#                 key = strain[2]
#                 values = [first_child[0].text]
#                 par = {key:values}
#             else:
#                 key = strain[2]
#                 values = None
#                 par = {key:values}       
#         except:
#             key = strain[2]
#             values = None
#             par = {key:values}
#     except:
#         key = strain[2]
#         values = None
#         par = {key:values}
    
        
#     flavors.append(par)

# df_flavors = pd.DataFrame(flavors)
# df_flavors.to_csv(filename4)  

# #collect parents of strains

# parents = []

# filename5 = 'parents.csv'

# for i in tqdm(range(0, len(soups_total))): 
#     children = soups_total[i].findAll('div',attrs={"class" : "strain-tile-footer"})
#     try:
#         if len(children) == 3:
#             key = children[0].text
#             values = [children[1].text, children[2].text]
#             par = {key:values}
#         elif len(children) == 2:
#             key = children[0].text
#             values = children[1].text
#             par = {key:values}        
#         else:
#             key = children[0].text
#             values = None
#             par = {key:values}
#     except:
#         None
#         #print("can't find that page")
    
#     parents.append(par)
   

# df_parents = pd.DataFrame(parents)
# df_parents.to_csv(filename5)

### Create Tokens

In [ ]:
category_tokens = ['hyrid', 'indica', 'sativa']

flavor_tokens = ['Ammonia', 'Apple', 'Apricot', 'Berry' 
                 'Blue Cheese', 'Blueberry', 'Butter', 
                 'Cheese', 'Chemical', 'Chestnut', 'Citrus',
                 'Coffee', 'Diesel', 'Earthy', 'Flowery',
                 'Grape', 'Grapefruit', 'Honey', 'Lavender',
                 'Lemon', 'Lime', 'Mango', 'Menthol', 'Mint',
                 'Nutty', 'Orange', 'Peach', 'Pear', 'Pepper',
                 'Pine', 'Pineapple', 'Plum', 'Pungent', 'Rose', 
                 'Sage', 'Skunk', 'Spicy/Herbal', 'Strawberry',
                 'Sweet', 'Tar', 'Tea', 'Tobacco', 'Tree Fruit',
                 'Tropical', 'Vanilla', 'Violet', 'Woody']

effect_tokens = ['Anxious', 'Aroused', 'Creative', 'Energetic',
                 'Euphoric', 'Focused', 'Giggly', 'Happy', 
                 'Hungry', 'Relaxed', 'Sleepy', 'Talkative',
                 'Tingly', 'Uplifted']

medical_tokens = ['Cramps', 'Depression', 'Eye Pressure', 'Fatigue',
                  'Headaches', 'Inflammation', 'Insomnia', 
                  'Lack Of Appetite', 'Muscle Spasms', 'Nausea',
                  'Pain Seizures', 'Spasticity', 'Stress', 'ADD/ADHD',
                  "Alzheimer's", 'Anorexia', 'Anxiety', 'Arthritis',
                  'Asthma', 'Bipolar Disorder', 'Cachexia', 'Cancer',
                  "Crohn's Disease", 'Epilepsy', 'Fibromyalgia', 
                  'Gastrointestinal Disorder', 'Glaucoma', 'HIV/AIDS',
                  'Hypertension', 'Migraines', 'Multiple Sclerosis',
                  'Muscular Dystrophy', 'PMS', 'PTSD', "Parkinson's",
                  'Phantom Limb Pain', 'Spinal Cord Injury',
                  'Tinnitus', "Tourette's Syndrome"]

In [ ]:
with open('soups_total.html', 'r') as f:
    html = f.read()
soups_total = BeautifulSoup(html,'html.parser')

### Import CSVs

In [3]:
#import categories - hybrid, sativa, indica
cat_df = pd.read_csv('strain_category.csv')

#import flavors
flav_df = pd.read_csv('flavors.csv')

#import parents
par_df = pd.read_csv('parents.csv')

#import effects
eff_df = pd.read_csv('effect.csv')

#import medical
med_df = pd.read_csv('med.csv')

#import negative
neg_df = pd.read_csv('neg.csv')

### Pivot Tables/Table Formatting

In [4]:
#drop first column
def drop_column(df):
    df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
drop_column(flav_df)
drop_column(par_df)
drop_column(eff_df)
drop_column(med_df)
drop_column(neg_df)
drop_column(cat_df)

In [6]:
#pivot tables so that column names are now indices
flav_series = flav_df.stack()
par_series = par_df.stack()
eff_series = eff_df.stack()
med_series = med_df.stack()
neg_series = neg_df.stack()

#convert from series to dataframe
flav_df = flav_series.to_frame(name='flavor')
par_df = par_series.to_frame(name='parent')
eff_df = eff_series.to_frame(name='effect')
med_df = med_series.to_frame(name='medical')
neg_df = neg_series.to_frame(name='negative')

#dataframe cleanup
def reset_index(df):
    df.reset_index(inplace=True)
    df.drop('level_0', axis=1, inplace=True)
    df.rename(columns={'level_1':'strain'}, inplace=True)

In [7]:
reset_index(flav_df)
reset_index(par_df)
reset_index(eff_df)
reset_index(med_df)
reset_index(neg_df)

In [8]:
#cat_df.reset_index('name', inplace=True)
cat_df.rename(columns={'strain':'category'}, inplace=True)
cat_df.rename(columns={'name':'strain'}, inplace=True)

#remove $ from parents table
par_df['strain'] = [x.strip('$') for x in par_df.strain]

#make all entries lowercase
flav_df = flav_df.apply(lambda x: x.astype(str).str.lower())
par_df = par_df.apply(lambda x: x.astype(str).str.lower())
eff_df = eff_df.apply(lambda x: x.astype(str).str.lower())
med_df = med_df.apply(lambda x: x.astype(str).str.lower())
neg_df = neg_df.apply(lambda x: x.astype(str).str.lower())
cat_df = cat_df.apply(lambda x: x.astype(str).str.lower())


#replace - with space in all tables
flav_df['strain'] = flav_df['strain'].str.replace('-',' ')
par_df['strain'] = par_df['strain'].str.replace('-',' ')
eff_df['strain'] = eff_df['strain'].str.replace('-',' ')
med_df['strain'] = med_df['strain'].str.replace('-',' ')
neg_df['strain'] = neg_df['strain'].str.replace('-',' ')
cat_df['strain'] = cat_df['strain'].str.replace('-',' ')


# #make all data lowercase
# def lowercase(df):
#     df = df.apply(lambda x: x.astype(str).str.lower())
#     return
    
# #replace dash with space in all tables
# def replace_dash(df, column):
#     df[column] = df[column].str.replace('-',' ')
#     return

In [9]:
def remove_char(df, column):
    df[column] = df[column].str.replace('[',' ')
    df[column] = df[column].str.replace(']',' ')
    df[column] = df[column].str.replace("'",'')
    

def new_columns_5(df, column):    
    
    # new data frame with split value columns 
    # making separate first flavor from new data frame 
    new1 = df[column].str.split(',', n = 1, expand = True)   
    df[str(column) + '_1'] = new1[0] 
    df[str(column) + '_2'] = new1[1]


    new2 = df[str(column) + '_2'].str.split(',', n = 1, expand = True)  
    df[str(column) + '_2']= new2[0] 
    df[str(column) + '_3']= new2[1] 
  
    new3 = df[str(column) + '_3'].str.split(',', n = 1, expand = True)
    df[str(column) + '_3']= new3[0]
    df[str(column) + '_4']= new3[1]


    new4 = df[str(column) + '_4'].str.split(',', n = 1, expand = True)
    df[str(column) + '_4']= new4[0]
    df[str(column) + '_5']= new4[1]


    #dropping old name columns 
    df.drop(columns =[column], inplace = True) 
    
def new_columns_3(df, column):    
    
    # new data frame with split value columns 
    # making separate first flavor from new data frame 
    new1 = df[column].str.split(',', n = 1, expand = True)   
    df[str(column) + '_1'] = new1[0] 
    df[str(column) + '_2'] = new1[1]


    new2 = df[str(column) + '_2'].str.split(',', n = 1, expand = True)  
    df[str(column) + '_2']= new2[0] 
    df[str(column) + '_3']= new2[1] 
  
    new3 = df[str(column) + '_3'].str.split(',', n = 1, expand = True)
    df[str(column) + '_3']= new3[0]

    #dropping old name columns 
    df.drop(columns =[column], inplace = True) 
    
def new_columns_2(df, column):    
    
    # new data frame with split value columns 
    # making separate first flavor from new data frame 
    new1 = df[column].str.split(',', n = 1, expand = True)   
    df[str(column) + '_1'] = new1[0] 
    df[str(column) + '_2'] = new1[1]

    #dropping old name columns 
    df.drop(columns =[column], inplace = True)

In [10]:
remove_char(flav_df, 'flavor')
remove_char(par_df, 'parent')
remove_char(eff_df, 'effect')
remove_char(med_df, 'medical')
remove_char(neg_df, 'negative')
remove_char(cat_df, 'category')

In [11]:
new_columns_5(eff_df, 'effect')
new_columns_5(med_df, 'medical')
new_columns_5(neg_df, 'negative')

In [12]:
new_columns_3(flav_df, 'flavor')
new_columns_2(par_df, 'parent')

In [13]:
par_df['strain'] = par_df['strain'].str.replace("'",'')

In [14]:
par_df.head(200)

,strain,parent_1,parent_2
0,100 og,og kush,None
1,12 year og,og kush,None
2,13 dawgs,g13,chemdawg
3,24k gold,kosher kush,tangie
4,3 bears og,triangle kush,None
5,100 og,og kush,None
6,12 year og,og kush,None
7,13 dawgs,g13,chemdawg
8,24k gold,kosher kush,tangie
9,3 bears og,triangle kush,None


### Create Final Merged Dataframe

In [16]:
#create properly formatted csvs

flav_final_df = pd.DataFrame(flav_df)
flav_final_df.to_csv('flav_final_df.csv')

par_final_df = pd.DataFrame(par_df)
par_final_df.to_csv('par_final_df.csv')

eff_final_df = pd.DataFrame(eff_df)
eff_final_df.to_csv('eff_final_df.csv')

med_final_df = pd.DataFrame(med_df)
med_final_df.to_csv('med_final_df.csv')

neg_final_df = pd.DataFrame(neg_df)
neg_final_df.to_csv('neg_final_df.csv')

cat_final_df = pd.DataFrame(cat_df)
cat_final_df.to_csv('cat_final_df.csv')

In [17]:
#import final csvs

flav_df = pd.read_csv('flav_final_df.csv')
par_df = pd.read_csv('par_final_df.csv')
eff_df = pd.read_csv('eff_final_df.csv')
med_df = pd.read_csv('med_final_df.csv')
neg_df = pd.read_csv('neg_final_df.csv')
cat_df = pd.read_csv('cat_final_df.csv')

In [18]:
#create master dataframe
drop_column(flav_df)
drop_column(par_df)
drop_column(eff_df)
drop_column(med_df)
drop_column(neg_df)
drop_column(cat_df)

final_df = pd.merge(cat_df, eff_df, how='outer')
final_df.drop_duplicates(inplace=True)
final_df = pd.merge(final_df, med_df, how='outer')
final_df.drop_duplicates(inplace=True)
final_df = pd.merge(final_df, neg_df, how='outer')
final_df.drop_duplicates(inplace=True)
final_df = pd.merge(final_df, flav_df, how='outer')
final_df.drop_duplicates(inplace=True)
final_df = pd.merge(final_df, par_df, how='outer')
final_df.drop_duplicates(inplace=True)

final_df.to_csv('final_df.csv')

In [19]:
final_df.head(50)

,category,strain,effect_1,effect_2,effect_3,effect_4,effect_5,medical_1,medical_2,medical_3,medical_4,medical_5,negative_1,negative_2,negative_3,negative_4,negative_5,flavor_1,flavor_2,flavor_3,parent_1,parent_2
0,hybrid,100 og,happy,uplifted,relaxed,euphoric,sleepy,stress,pain,depression,insomnia,lack of appetite,dry mouth,dizzy,dry eyes,anxious,paranoid,1. earthy,2. diesel,3. pungent,og kush,NaN
2,hybrid,98 white widow,creative,relaxed,energetic,sleepy,giggly,depression,muscle spasms,fatigue,pain,NaN,dry mouth,anxious,dizzy,dry eyes,headache,1. diesel,2. flowery,3. grape,NaN,NaN
3,sativa,1024,uplifted,energetic,happy,relaxed,focused,pain,fatigue,depression,stress,inflammation,dry mouth,dry eyes,dizzy,paranoid,anxious,1. tropical,2. mint,3. blueberry,NaN,NaN
4,hybrid,12 year og,focused,creative,hungry,euphoric,aroused,depression,pain,stress,NaN,NaN,anxious,dizzy,dry eyes,dry mouth,headache,1. diesel,NaN,NaN,og kush,NaN
6,hybrid,13 dawgs,sleepy,euphoric,focused,energetic,relaxed,pain,depression,stress,inflammation,fatigue,anxious,dry eyes,headache,dry mouth,NaN,1. diesel,2. apricot,3. citrus,g13,chemdawg
8,hybrid,22,energetic,focused,happy,uplifted,creative,pain,depression,fatigue,stress,muscle spasms,headache,NaN,NaN,NaN,NaN,1. diesel,2. pine,3. skunk,NaN,NaN
9,hybrid,24k gold,happy,relaxed,euphoric,uplifted,creative,stress,depression,pain,fatigue,insomnia,dry mouth,dry eyes,dizzy,anxious,headache,1. orange,2. citrus,3. sweet,kosher kush,tangie
11,indica,3 bears og,relaxed,giggly,sleepy,tingly,energetic,pain,stress,lack of appetite,muscle spasms,insomnia,dry mouth,dry eyes,headache,anxious,NaN,1. diesel,2. sweet,3. butter,triangle kush,NaN
13,hybrid,3 kings,happy,relaxed,uplifted,euphoric,energetic,stress,pain,depression,headaches,eye pressure,dry mouth,dry eyes,dizzy,headache,paranoid,1. earthy,2. diesel,3. pine,NaN,NaN
14,indica,303 og,focused,relaxed,uplifted,euphoric,sleepy,pain,depression,insomnia,muscle spasms,headaches,dry mouth,dry eyes,dizzy,anxious,NaN,1. diesel,2. earthy,3. woody,pre-98 bubba kush,chemdawg


In [20]:
len(final_df)

3073

In [80]:
urls = pd.read_csv('urls.csv')

In [194]:
urls_list = urls['url']

In [189]:
# #scrape individual strain pages

parents = []
reviews = []

for url in urls_list:
    html = requests.get(url).text
    soup = BeautifulSoup(html,'html.parser')

    parent = soup.findAll('div',attrs={"class" : "strain-tile-footer"})
    parents.append(parent)
    
    container = soup.find("a",{"class":"active"})  
    strain_name = container.get('href') 
    strain = strain_name.split('/')
    try: 
        review_total = soup.find('a',attrs={"onclick" : "window.strainHelpers.trackEvent('User Interactions', 'Strain Details', 'View all Reviews')"}).text
        review = review_total[review_total.find("(")+1:review_total.find(")")]
        rev_name = [strain[2], review]
    except:
        rev_name = [strain[2], None]
    
    reviews.append(rev_name)

time.sleep(1)

In [191]:
reviews

[['100-og', '509'], ['afghan-sour-kush', '17'], ['alien-og', '933']]

In [192]:
parents

[[<div class="strain-tile-footer">$100 OG</div>,
  <div class="strain-tile-footer">OG Kush</div>],
 [<div class="strain-tile-footer">Afghan Sour Kush</div>,
  <div class="strain-tile-footer">Afghani</div>,
  <div class="strain-tile-footer">Sour Kush</div>],
 [<div class="strain-tile-footer">Alien OG</div>,
  <div class="strain-tile-footer">Alien Kush</div>,
  <div class="strain-tile-footer">Tahoe OG Kush</div>]]

In [162]:
parents_final = []

for i in tqdm(range(0, len(parents))): 
    try:
        if len(parents[i]) == 7:
            key = parents[i][0].text
            values = [parents[i][1].text, parents[i][2].text,
                     parents[i][3].text, parents[i][4].text,
                     parents[i][5], parents[i][6]]
            par = {key:values}
        elif len(parents[i]) == 6:
            key = parents[i][0].text
            values = [parents[i][1].text, parents[i][2].text,
                 parents[i][3].text, parents[i][4].text,
                 parents[i][5]]
            par = {key:values}
        elif len(parents[i]) == 5:
            key = parents[i][0].text
            values = [parents[i][1].text, parents[i][2].text,
                 parents[i][3].text, parents[i][4].text]
            par = {key:values}
        elif len(parents[i]) == 4:
            key = parents[i][0].text
            values = [parents[i][1].text, parents[i][2].text,
                 parents[i][3].text]
            par = {key:values}    
        elif len(parents[i]) == 3:
            key = parents[i][0].text
            values = [parents[i][1].text, parents[i][2].text]
            par = {key:values}
        elif len(parents[i]) == 2:
            key = parents[i][0].text
            values = parents[i][1].text
            par = {key:values}        
        else:
            key = parents[i][0].text
            values = None
            par = {key:values}
    
        parents_final.append(par)
    
    except:
        None

In [193]:
parents_final

[{'$100 OG': 'OG Kush'},
 {'Afghan Sour Kush': ['Afghani', 'Sour Kush']},
 {'Alien OG': ['Alien Kush', 'Tahoe OG Kush']}]